# Embeddings reader

- Get mapping original number to raw ID
- Get splits
- Get embeddings of splits

In [1]:
# Reload modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

# Import from vm notebook dir
import sys; sys.path.insert(0, '../../')

import os
from datetime import datetime

from ExplainingDriftTextEmbeddings.access.file_storage import FileStorage
from ExplainingDriftTextEmbeddings.access.amazon_pickle_reader import AmazonPickleReader
from ExplainingDriftTextEmbeddings.access.interim_storage import InterimStorage

from BenchTest.amore.amazon_reviews_reader import AmazonReviewsReader

In [2]:
file_storage = FileStorage()

## Create reader

In [8]:
# Create AmazonPickleReader
amazon_raw_id = 'amazon_raw.pickle'
print(file_storage.get_filepath(amazon_raw_id))

amazon_directoy = os.path.dirname(file_storage.get_filepath(amazon_raw_id))
print(amazon_directoy)

amazon_reader = AmazonPickleReader(amazon_directoy)

# First call: Initialize mapping
#amazon_reader.get_raw_id(1)

/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/clustering/amazon_raw.pickle
/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/clustering


## Get mapping original number to raw ID

In [3]:
interim_storage = InterimStorage('amazon_originalno_to_rawid')

# Caching
if False:
    originalno_to_rawid = {}
    for originalno in amazon_reader.originalno_to_rawid.keys():
        originalno_to_rawid[originalno] = amazon_reader.get_raw_id(originalno)

    interim_storage.write(originalno_to_rawid)

    interim_storage.get_filepath()
    # '/tmp/InterimStorage/amazon_originalno_to_rawid.pickle.bz2'

# Use cache
else:
    originalno_to_rawid = interim_storage.read()

print(len(originalno_to_rawid))
# 7911684 -> all

7911684


## Get splits

In [4]:
# Swap to current root
import sys; sys.path.insert(0, '../../BenchTest')
from amore.reviews import Reviews
import pprint

In [5]:
def print_splits(file_id):
    splits = InterimStorage(file_id).read()
    pprint.pprint(splits)
    for split in splits:
        print(split.get_review_ids()[:5], '...')
    return splits

In [6]:
splits = print_splits('AMORE-IDs')

[Split(AMORE1,A,10000),
 Split(AMORE1,B,9900),
 Split(AMORE1,B,100),
 Split(AMORE2,A,10000),
 Split(AMORE2,B,9800),
 Split(AMORE2,B,200)]
[334241, 705914, 1070406, 1206648, 1229742] ...
[3569722, 3570714, 3571533, 3571568, 3577009] ...
[3116450, 3239880, 4268728, 4606075, 709244] ...
[1707082, 7418424, 270959, 3956447, 897306] ...
[2473162, 2473326, 2473342, 2473508, 2474416] ...
[623398, 712539, 934046, 940102, 1095175] ...


## Get embeddings

In [25]:
first_split_index = 0
last_split_index = 2
results = {}
for i in range(first_split_index,last_split_index+1):
    split = splits[i]
    print(split)
    embeddings = []
    for review_no in split.get_review_ids():
        raw_id = originalno_to_rawid[review_no]
        embeddings.append(amazon_reader.get_bow50(raw_id))
    results[i] = embeddings

Split(AMORE1,A,10000)
Split(AMORE1,B,9900)
Split(AMORE1,B,100)


In [29]:
for i in results:
    print(i, len(results[i]), len(results[i][0]))

0 10000 50
1 9900 50
2 100 50


In [30]:
results_storage = InterimStorage('tmp_embeddings_list').write(results)